In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('Dataset.csv',encoding='ISO-8859-1')

In [3]:
df.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [4]:
df = df[['target','text']]

In [5]:
df['target'] = df['target'].replace(4,1)

In [6]:
df.tail()

,target,text
19995,1,Just woke up. Having no school is the best fee...
19996,1,TheWDB.com - Very cool to hear old Walt interv...
19997,1,Are you ready for your MoJo Makeover? Ask me f...
19998,1,Happy 38th Birthday to my boo of alll time!!! ...
19999,1,happy #charitytuesday @theNSPCC @SparksCharity...


In [7]:
df.isnull().sum()

target    0
text      0
dtype: int64

In [8]:
df['target'].value_counts()

target
0    10000
1    10000
Name: count, dtype: int64

In [9]:
new_df = df.copy()

In [10]:
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [11]:
port_stem = PorterStemmer()

In [12]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)

    return stemmed_content

In [13]:
new_df['stemmed_text'] = new_df['text'].apply(stemming)

In [14]:
new_df

,target,text,stemmed_text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see
...,...,...,...
19995,1,Just woke up. Having no school is the best fee...,woke school best feel ever
19996,1,TheWDB.com - Very cool to hear old Walt interv...,thewdb com cool hear old walt interview http b...
19997,1,Are you ready for your MoJo Makeover? Ask me f...,readi mojo makeov ask detail
19998,1,Happy 38th Birthday to my boo of alll time!!! ...,happi th birthday boo alll time tupac amaru sh...


In [15]:
new_df.drop(columns=['text'],inplace=True)

In [16]:
new_df

,target,stemmed_text
0,0,switchfoot http twitpic com zl awww bummer sho...
1,0,upset updat facebook text might cri result sch...
2,0,kenichan dive mani time ball manag save rest g...
3,0,whole bodi feel itchi like fire
4,0,nationwideclass behav mad see
...,...,...
19995,1,woke school best feel ever
19996,1,thewdb com cool hear old walt interview http b...
19997,1,readi mojo makeov ask detail
19998,1,happi th birthday boo alll time tupac amaru sh...


In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()

In [18]:
X = tfidf.fit_transform(new_df['stemmed_text']).toarray()

In [19]:
Y = new_df['target'].values

In [20]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.2,random_state=2)

In [21]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression()

In [22]:
LR.fit(x_train,y_train)

LogisticRegression()

In [23]:
y_pred = LR.predict(x_test)

In [24]:
from sklearn.metrics import accuracy_score,confusion_matrix,precision_score

In [25]:
print(accuracy_score(y_test,y_pred))
print(precision_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

0.7505
0.7429378531073446
[[1424  546]
 [ 452 1578]]


In [26]:
import pickle
pickle.dump(tfidf,open('vectorizer.pkl','wb'))
pickle.dump(LR,open('model.pkl','wb'))